Using the `all data` gdb of historical fires from Calfire:
https://www.fire.ca.gov/what-we-do/fire-resource-assessment-program/fire-perimeters

In [3]:
import requests
import zipfile
import os
import fiona
import geopandas as gpd

# Extract zip file
with zipfile.ZipFile("fire221gdb.zip", "r") as zip_ref:
    zip_ref.extractall(".")

# Get the name of the gdb file
gdb_file = [f for f in os.listdir(".") if f.endswith(".gdb")][0]

# List all layers in the GDB file
layers = fiona.listlayers(gdb_file)

According to calfire:

```
The fire perimeters database includes three layers—historical fire perimeters (firep), Rx treatments using fire (rxburn), and Rx treatments not using fire (Non_RXFire_Legacy).
```

In [4]:
layers

['rxburn22_1', 'firep22_1', 'Non_RXFire_Legacy13_2']

In [28]:
rx_treatments_with_fire = gpd.read_file(gdb_file, layer=layers[0])
rx_treatments_without_fire = gpd.read_file(gdb_file, layer=layers[2])
fire_perimeters = gpd.read_file(gdb_file, layer=layers[1])

rx_treatments_with_fire.to_crs(epsg=4326, inplace=True)
rx_treatments_without_fire.to_crs(epsg=4326, inplace=True)
fire_perimeters.to_crs(epsg=4326, inplace=True)

In [29]:
rx_treatments_with_fire.head()

,YEAR_,STATE,AGENCY,UNIT_ID,TREATMENT_ID,TREATMENT_NAME,START_DATE,END_DATE,TREATED_AC,GIS_ACRES,RX_CONSUM,PRE_CON_CLASS,POST_CON_CLASS,TREATMENT_TYPE,Shape_Length,Shape_Area,geometry
0,2020,CA,CDF,AEU,10509,Jan 31 2020 Broadcast,2020-01-31 00:00:00+00:00,2020-01-31T00:00:00+00:00,15.0,15.047042,NaN,NaN,NaN,1.0,1365.887529,60893.216906,"MULTIPOLYGON (((-120.52136 38.75171, -120.5212..."
1,2020,CA,PVT,AEU,10572,2020 Fuels Reduction,2020-02-13 00:00:00+00:00,2020-02-13T00:00:00+00:00,22.3,36.739700,NaN,NaN,NaN,1.0,2681.610336,148680.293398,"MULTIPOLYGON (((-120.66046 38.90069, -120.6605..."
2,2020,CA,CDF,AEU,10630,Feb 2020 Broadcast,2020-02-13 00:00:00+00:00,2020-02-18T00:00:00+00:00,22.4,38.839832,NaN,NaN,NaN,1.0,3918.219956,157179.228865,"MULTIPOLYGON (((-120.57195 38.50208, -120.5718..."
3,2020,CA,CDF,AEU,10639,Feb 2020 Broadcast,2020-02-11 00:00:00+00:00,2020-02-20T00:00:00+00:00,75.5,75.498100,NaN,NaN,NaN,1.0,3918.834462,305529.980211,"MULTIPOLYGON (((-120.54192 38.74314, -120.5419..."
4,2020,CA,CDF,AEU,10780,Mar 3 2020 Broadcast,2020-03-03 00:00:00+00:00,2020-03-03T00:00:00+00:00,61.8,61.771519,NaN,NaN,NaN,1.0,2769.034316,249980.461716,"MULTIPOLYGON (((-120.54811 38.74108, -120.5481..."


In [30]:
rx_treatments_without_fire.head()

,TREATMENT_ID,TREATMENT_NAME,TREATMENT_TYPE,UNIT_ID,AGENCY,TREATED_AC,GIS_ACRES,STATE,YEAR_,RX_CONSUM,PRE_CON_CLASS,POST_CON_CLASS,END_DATE,START_DATE,Shape_Length,Shape_Area,geometry
0,3277604,NON_WUI,13,SQF,USF,136.388878,136.388840,CA,2006,NaN,NaN,NaN,2006-04-15 00:00:00+00:00,NaT,7391.270964,5.519460e+05,"MULTIPOLYGON (((-118.50884 36.10266, -118.5086..."
1,3277608,NON_WUI,15,SQF,USF,292.127163,292.126648,CA,2006,NaN,NaN,NaN,2006-07-03 00:00:00+00:00,NaT,17166.848106,1.182195e+06,"MULTIPOLYGON (((-118.96349 36.78626, -118.9632..."
2,3277612,SOUTHRIDGE 1-1 CHIP,5,BDF,USF,4.726939,4.726954,CA,2006,NaN,NaN,NaN,2006-02-01 00:00:00+00:00,NaT,880.091816,1.912930e+04,"MULTIPOLYGON (((-116.69961 33.73683, -116.6997..."
3,3277613,SOUTHRIDGE UNIT 5 CUT AND STACK,9,BDF,USF,22.742776,22.742844,CA,2006,NaN,NaN,NaN,2006-03-01 00:00:00+00:00,NaT,1623.896026,9.203702e+04,"MULTIPOLYGON (((-116.70554 33.72558, -116.7052..."
4,3277840,HIGHWAY 20 CUT & PILING (06),9,TNF,USF,106.348339,106.347382,CA,2006,NaN,NaN,NaN,2006-09-15 00:00:00+00:00,NaT,3002.279795,4.303726e+05,"MULTIPOLYGON (((-120.79859 39.31701, -120.7983..."


In [31]:
fire_perimeters.head()

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry
0,2020,CA,CDF,NEU,NELSON,00013212,2020-06-18T00:00:00+00:00,2020-06-23T00:00:00+00:00,11.0,,109.602280,1.0,1.0,None,None,None,None,3252.523280,4.435447e+05,"MULTIPOLYGON (((-121.34841 38.88999, -121.3483..."
1,2020,CA,CDF,NEU,AMORUSO,00011799,2020-06-01T00:00:00+00:00,2020-06-04T00:00:00+00:00,2.0,,685.585022,1.0,1.0,None,None,None,None,9653.760308,2.774464e+06,"MULTIPOLYGON (((-121.35275 38.82039, -121.3526..."
2,2020,CA,CDF,NEU,ATHENS,00018493,2020-08-10T00:00:00+00:00,2020-08-11T00:00:00+00:00,14.0,,27.300480,1.0,1.0,None,None,None,None,1649.643235,1.104811e+05,"MULTIPOLYGON (((-121.33334 38.84558, -121.3331..."
3,2020,CA,CDF,NEU,FLEMING,00007619,2020-03-31T00:00:00+00:00,2020-04-01T00:00:00+00:00,9.0,,12.931545,1.0,1.0,None,None,None,None,1577.155857,5.233211e+04,"MULTIPOLYGON (((-121.27317 38.96308, -121.2730..."
4,2020,CA,CDF,NEU,MELANESE,00008471,2020-04-14T00:00:00+00:00,2020-04-19T00:00:00+00:00,18.0,,10.315964,1.0,1.0,None,None,None,None,1035.787625,4.174722e+04,"MULTIPOLYGON (((-121.30066 39.48714, -121.3004..."


In [32]:
fire_perimeters[fire_perimeters["AGENCY"] == "NPS"]

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry
276,2020,CA,NPS,MNP,BULL,00013423,2020-09-05T00:00:00+00:00,2020-09-05T00:00:00+00:00,10.0,AFC018BF-96ED-4018-AB90-77E625432EA7,14.089692,8.0,1.0,None,None,None,None,1655.085138,5.701896e+04,"MULTIPOLYGON (((-115.79681 35.40100, -115.7961..."
305,2020,CA,NPS,YNP,BLUEJAY,00000054,2020-07-25T00:00:00+00:00,2020-11-19T00:00:00+00:00,1.0,,6922.013672,7.0,2.0,None,None,None,None,44088.647246,2.801240e+07,"MULTIPOLYGON (((-119.60746 37.81324, -119.6074..."
306,2020,CA,NPS,BNP,CALDWELL,00000479,2020-07-22T00:00:00+00:00,2020-09-01T00:00:00+00:00,1.0,,81224.679688,7.0,1.0,None,None,None,None,176044.495381,3.287046e+08,"MULTIPOLYGON (((-121.60456 41.83603, -121.6045..."
307,2020,CA,NPS,MNP,DOME,00012356,2020-08-15T00:00:00+00:00,2020-09-12T00:00:00+00:00,1.0,Auto-generated by EGP-IRWIN,44211.250000,8.0,1.0,None,None,None,None,76882.528186,1.789166e+08,"MULTIPOLYGON (((-115.54279 35.36503, -115.5461..."
308,2020,CA,NPS,YNP,HORSE,00000089,2020-08-23T00:00:00+00:00,2020-11-19T00:00:00+00:00,1.0,,30.192495,1.0,2.0,None,None,None,None,4215.009845,1.221847e+05,"MULTIPOLYGON (((-119.53190 37.62689, -119.5319..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21856,2022,CA,NPS,YNP,RED,00000056,2022-08-04T00:00:00+00:00,2022-11-06T00:00:00+00:00,1.0,None,8432.418945,3.0,2.0,None,None,None,1E643C3B-3BE4-4669-892B-CF82B46FA820,98931.614579,3.412479e+07,"MULTIPOLYGON (((-119.48282 37.70949, -119.4828..."
21857,2022,CA,NPS,YNP,RODGERS,00000058,2022-08-08T00:00:00+00:00,2022-11-06T00:00:00+00:00,1.0,None,2839.684082,3.0,2.0,None,None,None,99B90DBC-6F1A-4494-B5C6-9CFDF335492A,54091.789770,1.149179e+07,"MULTIPOLYGON (((-119.52391 37.94608, -119.5239..."
21858,2022,CA,NPS,KNP,AVALANCHE,00000049,2022-07-18T00:00:00+00:00,2022-11-10T00:00:00+00:00,1.0,None,28.925026,7.0,2.0,None,None,None,52D1B9EC-9479-48E6-AF4A-D1550E8A97CB,1415.510211,1.170554e+05,"MULTIPOLYGON (((-118.58141 36.77579, -118.5815..."
21859,2022,CA,NPS,KNP,SUMMIT,00000075,2022-08-03T00:00:00+00:00,2022-10-19T00:00:00+00:00,1.0,None,1394.439575,7.0,2.0,None,None,None,F3705FA2-72E3-4E2A-BBE0-EA04D37FAE54,15630.965622,5.643097e+06,"MULTIPOLYGON (((-118.66634 36.33272, -118.6663..."


To batch process (some) of these, we can use our batch endpoint, which accepts all of the form arguments we get from the frontend and does our anaylze and fetch workflows. 

In [33]:
import sys
sys.path.append("..")

from src.routers.dependencies import get_cloud_logger, get_cloud_static_io_client 

from dotenv import load_dotenv
load_dotenv("../.devcontainer/.env")

from google.cloud import tasks_v2
from google.protobuf import timestamp_pb2
import geopandas as gpd
import datetime
import json
from shapely.geometry import box
from src.routers.batch.batch_analyze_and_fetch import main as batch_analyze_and_fetch
from google.cloud import logging
from src.util.cloud_static_io import CloudStaticIOClient

# Create a client
client = tasks_v2.CloudTasksClient()

# Get logger
# logging_client = logging.Client(project="dse-nps")
# log_name = "burn-backend"
# logger = logging_client.logger(log_name)
logger = get_cloud_logger()

# Get cloud static io client
# s3_bucket_name = os.getenv("S3_BUCKET_NAME")
# cloud_static_io_client = CloudStaticIOClient(s3_bucket_name, "s3")
cloud_static_io_client = get_cloud_static_io_client(logger=logger)

# Define our Google Cloud Project ID and Queue ID
project = "dse-nps"
queue = "tf-rest-burn-severity-queue-dev"
location = "us-central1"

# Define the URL endpoint of our cloud function
dev_url = "https://tf-rest-burn-severity-dev-ohi6r6qs2a-uc.a.run.app"

# Convert DataFrame row to dictionary
def row_to_dict(row):
    return row.to_dict()


# Create a task with row data as payload
def create_task(row):
    # Convert the row to a dictionary
    row_dict = row_to_dict(row)

    post_body = {
        "fire_event_name": row_dict["FIRE_NAME"],
        "affiliation": row_dict["AGENCY"],
        "ignition_date": row_dict["IGNITION_DA"],
        "containment_date": row_dict["CONTAINMEN"],
        "time_buffer_days": 30,
        "derive_boundary": False,
        "geojson_boundary" : row_dict["geometry"].__geo_interface__
    }

    batch_analyze_and_fetch(
        geojson_boundary=post_body["geojson_boundary"],
        fire_event_name=post_body["fire_event_name"],
        affiliation=post_body["affiliation"],
        derive_boundary=post_body["derive_boundary"],
        logger=logger,
        cloud_static_io_client=cloud_static_io_client,
        ignition_date=post_body["ignition_date"],
        containment_date=post_body["containment_date"],
        time_buffer_days=post_body["time_buffer_days"]
    )
    
    # # Convert the dictionary to a JSON string
    # payload = json.dumps(row_dict).encode()

    # # Construct the request body
    # parent = f"projects/{project}/locations/{location}/queues/{queue}"
    # task = {
    #     "http_request": {
    #         "http_method": "POST",
    #         "url": dev_url,
    #         "body": payload,
    #         "headers": {"Content-type": "application/json"},
    #     }
    # }

    # # Add the timestamp to the tasks
    # timestamp = timestamp_pb2.Timestamp()
    # timestamp.FromDatetime(datetime.datetime.utcnow() + datetime.timedelta(seconds=10))
    # task["schedule_time"] = timestamp

    # # Use the client to build and send the task
    # response = client.create_task(request={"parent": parent, "task": task})

    # print("Created task {}".format(response.name))

/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/burn-severity/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your applica

For our test, we want Southern CA fires during Sentinel 2's time:

In [40]:
lat_min = 32
lon_min = -120
lat_max = 37
lon_max = -115

# make into a polygon geopandas for sjoin
bbox = box(lon_min, lat_min, lon_max, lat_max)
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")
bbox_gdf

,geometry
0,"POLYGON ((-115.00000 32.00000, -115.00000 37.0..."


In [41]:
bbox.bounds

(-120.0, 32.0, -115.0, 37.0)

In [42]:
from geopandas.tools import sjoin
sample_fire_perimeter = sjoin(fire_perimeters, bbox_gdf, how="inner")
sample_fire_perimeter["YEAR_"] > 2016

In [43]:
sample_fire_perimeter

,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,...,C_METHOD,OBJECTIVE,FIRE_NUM,COMPLEX_NAME,COMPLEX_INCNUM,IRWINID,Shape_Length,Shape_Area,geometry,index_right
38,2020,CA,CCO,ORC,PARK FIRE,20075598,2020-07-05T00:00:00+00:00,2020-07-07T00:00:00+00:00,14.0,,...,1.0,1.0,None,None,None,None,1676.332591,9.752050e+04,"MULTIPOLYGON (((-117.74447 33.79613, -117.7444...",0
39,2020,CA,CDF,ORC,SILVERADO,20121364,2020-10-26T00:00:00+00:00,2020-11-02T00:00:00+00:00,14.0,,...,7.0,1.0,None,None,None,None,54296.829137,5.046109e+07,"MULTIPOLYGON (((-117.71344 33.74971, -117.7124...",0
40,2020,CA,CDF,ORC,BLUE RIDGE,20121612,2020-10-26T00:00:00+00:00,2020-11-02T00:00:00+00:00,14.0,,...,7.0,1.0,None,None,None,None,54580.299599,5.541981e+07,"MULTIPOLYGON (((-117.73679 33.94161, -117.7364...",0
41,2020,CA,CDF,ORC,BOND,20136890,2020-12-03T00:00:00+00:00,2020-12-07T00:00:00+00:00,14.0,,...,7.0,1.0,None,None,None,None,43845.910280,2.703745e+07,"MULTIPOLYGON (((-117.63795 33.74450, -117.6372...",0
59,2020,CA,CCO,KRN,BAKER,02024545,2020-06-17T00:00:00+00:00,2020-06-18T00:00:00+00:00,14.0,None,...,1.0,1.0,None,None,None,None,6175.438813,6.684587e+05,"MULTIPOLYGON (((-118.69154 35.31233, -118.6894...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21918,2022,CA,USF,LPF,HOWARD 2,00002477,2022-10-08T00:00:00+00:00,2022-10-10T00:00:00+00:00,9.0,FIRIS captured perimeter.,...,2.0,1.0,None,None,None,04745F7C-3F8B-47A4-8EFE-C8F3C84742D9,3179.585169,3.019557e+05,"MULTIPOLYGON (((-119.12967 34.55472, -119.1296...",0
21919,2022,CA,CCO,VNC,LYNN,00105122,2022-11-17T00:00:00+00:00,2022-11-17T00:00:00+00:00,14.0,None,...,1.0,1.0,None,None,None,94396A63-3D17-4429-98CC-103F9D85833B,790.145569,2.331727e+04,"MULTIPOLYGON (((-118.98808 34.15615, -118.9881...",0
21920,2022,CA,CCO,VNC,ANA,00077086,2022-08-25T00:00:00+00:00,2022-08-25T00:00:00+00:00,14.0,"FIRIS aircraft, INTEL-24",...,3.0,1.0,None,None,None,31644E00-C263-4B4C-A373-4DCE7FA2831D,391.612566,7.452293e+03,"MULTIPOLYGON (((-119.31984 34.39386, -119.3197...",0
21921,2022,CA,LRA,VNC,CHAMBERS,00023419,2022-03-16T00:00:00+00:00,2022-03-17T00:00:00+00:00,14.0,None,...,1.0,1.0,None,None,None,B541FD2D-0474-47CF-AF6C-C4576E03A590,443.298121,6.261644e+03,"MULTIPOLYGON (((-118.91737 34.39068, -118.9173...",0


In [1]:
# Iterate over DataFrame rows and create a task for each
for index, row in sample_fire_perimeter.iterrows():
    create_task(row)

NameError: name 'sample_fire_perimeter' is not defined